# 메타데이터 전처리
카카오아레나의 'Melon Playlist Continuation'대회에서 제공하는 데이터 셋 중 song_meta.json파일과 genre_gn_all.json파일의 '가수','앨범','노래제목'을 기반으로 가사 크롤링을 하기전에, 가사가 없는 곡들을 제거하고 메타 데이터를 살펴본다.

In [1]:
import bs4
import re
import json
import time
import numpy as np
import pandas as pd

In [2]:
## Metadata and genre information load
def json_to_tsv(json_file, tsv_file):
    with open(json_file, encoding="UTF-8") as f:
        js = json.load(f)
        if isinstance(js, list):
            df = pd.DataFrame.from_records(js)
            df['song_gn_gnr_basket'] = df['song_gn_gnr_basket'].apply(lambda x : ','.join(x))
            df['song_gn_dtl_gnr_basket'] = df['song_gn_dtl_gnr_basket'].apply(lambda x : ','.join(x))
            df['artist_name_basket'] = df['artist_name_basket'].apply(lambda x : ','.join(x))
            df['artist_id_basket'] = df['artist_id_basket'].apply(lambda x : str(x)[1:-1])
        if isinstance(js, dict):
            df = pd.DataFrame.from_dict([js])
            df = df.transpose()
            df.reset_index(inplace = True)
            df.columns = ['genre_id', 'genre']
        df.to_csv(tsv_file, sep = '\t', index = None, encoding = 'utf-8')
    return df

In [3]:
def preprocessing(meta, genre : pd.DataFrame):
    
    #### genre filtering ####
    genre.columns = ['song_gn_gnr_basket', 'genre']
    '''
    GN1100	일렉트로니카
    GN1600	클래식
    GN1700	재즈
    GN1800	뉴에이지
    GN1900	J-POP
    GN2400	국악
    GN2600	일렉트로니카
    GN2700	EDM
    GN2800	뮤직테라피
    
    GN2000	월드뮤직
    GN2100	CCM
    GN2200	어린이/태교
    GN2300	종교음악
    GN2900	뮤지컬
    GN3000	크리스마스


    '''
    instrumental_music = ['GN1100', 'GN1600', 'GN1700', 'GN1800', 'GN1900', 'GN2000', 'GN2100', 'GN2200', 'GN2300', 'GN2400', 'GN2600', 'GN2700', 'GN2800','GN2900', 'GN3000']
    # Main genre
    main_genre = genre[genre['song_gn_gnr_basket'].str.contains('^.*00$', regex=True)]
    # Omitting the genre of instrumental music 
    ss_lyric_genre = main_genre[~main_genre['song_gn_gnr_basket'].isin(instrumental_music)]
    # Remove instrumental music among song with single and multi genre in metadata
    meta = meta[ np.vectorize(lambda x : bool(set(x.split(',')) & set(ss_lyric_genre['song_gn_gnr_basket']) ))(meta['song_gn_gnr_basket'])]
    meta.reset_index(drop = True, inplace = True)
    
    
    
    #### html special symbol filtering ####
    song_index = meta[meta['song_name'].str.contains("^.*(&#).*", regex= True, na = False)].index#6015
    album_index = meta[meta['album_name'].str.contains("^.*(&#).*", regex= True, na = False)].index#3320
    artist_index = meta[meta['artist_name_basket'].str.contains("^.*(&#).*", regex= True, na = False)].index#327

    meta.loc[song_index, 'song_name'] \
    = meta.loc[song_index, 'song_name']\
    .apply(lambda x : bs4.BeautifulSoup(x, 'html.parser'))\
    .apply(lambda x : ','.join(x))

    meta.loc[album_index, 'album_name']\
    = meta.loc[album_index, 'album_name']\
    .apply(lambda x : bs4.BeautifulSoup(x, 'html.parser'))\
    .apply(lambda x : ','.join(x))

    meta.loc[artist_index, 'artist_name_basket']\
    = meta.loc[artist_index, 'artist_name_basket']\
    .apply(lambda x : bs4.BeautifulSoup(x, 'html.parser'))\
    .apply(lambda x : ','.join(x))
    
    meta = meta[~meta['song_name'].str.contains("inst", flags = re.I, na = False)]
    meta = meta[~meta['song_name'].str.contains("(대사)(\)|\s)", regex = True, na = False)]
    
    
    meta.to_csv('metadata_filtered.tsv', sep = '\t', index = None, encoding = 'utf-8')
    return meta

In [4]:
start = time.time()
meta = json_to_tsv("song_meta.json", "song_meta.tsv")
genre = json_to_tsv("genre_gn_all.json", "genre_gn_all.tsv")
meta = preprocessing(meta, genre)
print("time :", time.time() - start)

C:\Users\user\AppData\Local\Temp\ipykernel_7032\1431309541.py:37: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  song_index = meta[meta['song_name'].str.contains("^.*(&#).*", regex= True, na = False)].index#6015
C:\Users\user\AppData\Local\Temp\ipykernel_7032\1431309541.py:38: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  album_index = meta[meta['album_name'].str.contains("^.*(&#).*", regex= True, na = False)].index#3320
C:\Users\user\AppData\Local\Temp\ipykernel_7032\1431309541.py:39: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  artist_index = meta[meta['artist_name_basket'].str.contains("^.*(&#).*", regex= True, na = False)].index#327
C:\Users\user\Link\resources\mrx-link-venv\lib\site-packages\bs4\__init__.py:435: Markup

time : 28.18452501296997


In [5]:
meta = pd.read_csv("metadata_filtered.tsv", sep = '\t')
meta

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
0,GN0901,20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,2727,Feelings,GN0900,Various Artists,0
1,GN0901,20180518,Hit,4698747,3361,Solsbury Hill (Remastered 2002),GN0900,Peter Gabriel,2
2,GN0301,20070625,Sexualmetro,353020,224583,Lovers’ Leap (Feat. Qypthone),GN0300,애플스,8
3,"GN0105,GN0101",20170320,Pastel Reflection,10047088,753752,"사랑, 그대라는 멜로디",GN0100,진호,9
4,GN1201,20170407,Luv.Loops,10053652,1625859,Hi (Heyoo),GN1200,Miraa.,10
...,...,...,...,...,...,...,...,...,...
410905,"GN2207,GN1501,GN1506,GN1509",20160601,생일왕국의 프린세스 프링 OST1 : 프린세스 프링의 초대,2688257,889414,생일축하노래,"GN1500,GN2200",ButterFly,707981
410906,"GN1502,GN1501",20080929,고고 70 OST,398382,258169,밤차 (Feat. 신민아),GN1500,조승우와 데블스,707982
410907,GN0901,19860000,True Colors,44141,11837,Change Of Heart,GN0900,Cyndi Lauper,707985
410908,"GN0105,GN0101",20160120,행보 2015 윤종신 / 작사가 윤종신 Live Part.1,2662866,437,스치듯 안녕,GN0100,윤종신,707986


# 크롤링한 데이터 전처리

### Merge all metadata

In [6]:
import os

meta_list = os.listdir('lyric_2')
meta_list

['meta_lyric_0to10000.tsv',
 'meta_lyric_100000to105000.tsv',
 'meta_lyric_10000to20000.tsv',
 'meta_lyric_105000to110000.tsv',
 'meta_lyric_110000to115000.tsv',
 'meta_lyric_115000to120000.tsv',
 'meta_lyric_120000to125000.tsv',
 'meta_lyric_125000to130000.tsv',
 'meta_lyric_130000to135000.tsv',
 'meta_lyric_135000to140000.tsv',
 'meta_lyric_140000to145000.tsv',
 'meta_lyric_145000to150000.tsv',
 'meta_lyric_150000to155000.tsv',
 'meta_lyric_155000to160000.tsv',
 'meta_lyric_160000to170000.tsv',
 'meta_lyric_170000to180000.tsv',
 'meta_lyric_180000to190000.tsv',
 'meta_lyric_190000to200000.tsv',
 'meta_lyric_200000to210000.tsv',
 'meta_lyric_20000to30000.tsv',
 'meta_lyric_210000to220000.tsv',
 'meta_lyric_220000to230000.tsv',
 'meta_lyric_230000to240000.tsv',
 'meta_lyric_240000to250000.tsv',
 'meta_lyric_250000to260000.tsv',
 'meta_lyric_260000to264500.tsv',
 'meta_lyric_264500to269000.tsv',
 'meta_lyric_269000to270000.tsv',
 'meta_lyric_270000to280000.tsv',
 'meta_lyric_280000to290

In [7]:
# 새로운 데이터 프레임 생성
df_all_years = pd.DataFrame()

for metas in meta_list:
    df= pd.read_csv("./lyric_2/" +metas, sep = '\t')
    df_all_years = pd.concat([df_all_years, df])

In [8]:
print(len(df_all_years))
print(len(meta))

410910
410910


In [9]:
df_all_years.reset_index(drop = True, inplace = True)

In [10]:
meta = df_all_years

### Remove html special symbol

In [11]:
#### html special symbol filtering ####
song_index = meta[meta['song_name'].str.contains("^.*(&#).*", regex= True, na = False)].index#6015
album_index = meta[meta['album_name'].str.contains("^.*(&#).*", regex= True, na = False)].index#3320
artist_index = meta[meta['artist_name_basket'].str.contains("^.*(&#).*", regex= True, na = False)].index#327

meta.loc[song_index, 'song_name'] \
= meta.loc[song_index, 'song_name']\
.apply(lambda x : bs4.BeautifulSoup(x, 'html.parser'))\
.apply(lambda x : ','.join(x))

meta.loc[album_index, 'album_name']\
= meta.loc[album_index, 'album_name']\
.apply(lambda x : bs4.BeautifulSoup(x, 'html.parser'))\
.apply(lambda x : ','.join(x))

meta.loc[artist_index, 'artist_name_basket']\
= meta.loc[artist_index, 'artist_name_basket']\
.apply(lambda x : bs4.BeautifulSoup(x, 'html.parser'))\
.apply(lambda x : ','.join(x))

C:\Users\user\AppData\Local\Temp\ipykernel_7032\930040905.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  song_index = meta[meta['song_name'].str.contains("^.*(&#).*", regex= True, na = False)].index#6015
C:\Users\user\AppData\Local\Temp\ipykernel_7032\930040905.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  album_index = meta[meta['album_name'].str.contains("^.*(&#).*", regex= True, na = False)].index#3320
C:\Users\user\AppData\Local\Temp\ipykernel_7032\930040905.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  artist_index = meta[meta['artist_name_basket'].str.contains("^.*(&#).*", regex= True, na = False)].index#327
C:\Users\user\Link\resources\mrx-link-venv\lib\site-packages\bs4\__init__.py:435: MarkupResemb

### Remove without lyrics 

In [12]:
## Without lyric
meta = meta[meta['lyric'] != 'adult or None']
meta = meta[meta['lyric'] != 'Page not found']
meta = meta[meta['lyric'] != 'Not found']
meta = meta[meta['lyric'] != '-']
meta = meta[~meta['song_name'].str.contains("inst", flags = re.I, na = False)]  #7660

##lyric = re.sub("�","", lyric)

## 드라마 또는 영화 대사
meta = meta[~meta['song_name'].str.contains("(대사)(\)|\s)", regex = True, na = False)]
meta.reset_index(drop = True, inplace = True)
len(meta)

C:\Users\user\AppData\Local\Temp\ipykernel_7032\3416230646.py:11: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  meta = meta[~meta['song_name'].str.contains("(대사)(\)|\s)", regex = True, na = False)]


206936

### Remove very short lyric

In [13]:
## 가사 길이가 비정상적으로 짧은 노래들
meta = meta[meta['lyric'].apply(lambda x : len(x) > 50)]
len(meta)

206807

### 가사 전처리 시작

In [16]:
meta = meta[~meta['lyric'].str.contains("verse|hook|CHORUS|BRIDGE|PreChorus|outro|intro|x(\d)|�|http|\)|\:|\[|\(|\]", flags = re.I, na = False)]

C:\Users\user\AppData\Local\Temp\ipykernel_7032\2891014722.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  meta = meta[~meta['lyric'].str.contains("verse|hook|CHORUS|BRIDGE|PreChorus|outro|intro|x(\d)|�|http|\)|\:|\[|\(|\]", flags = re.I, na = False)]


In [17]:
print(len(meta[meta['lyric'].str.contains("\[", flags = re.I, na = False)]))
print(len(meta[meta['lyric'].str.contains("\(", flags = re.I, na = False)]))
print(len(meta[meta['lyric'].str.contains("\:", flags = re.I, na = False)]))
print(len(meta[meta['lyric'].str.contains("verse|hook|CHORUS|BRIDGE|PreChorus|outro|intro|x(\d)|�|http|\)|\:|\[|\(|\]", flags = re.I, na = False)]))

0
0
0


C:\Users\user\AppData\Local\Temp\ipykernel_7032\315461593.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  print(len(meta[meta['lyric'].str.contains("verse|hook|CHORUS|BRIDGE|PreChorus|outro|intro|x(\d)|�|http|\)|\:|\[|\(|\]", flags = re.I, na = False)]))


0


In [18]:
meta

,index,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,lyric,song_id
0,0,GN0901,20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,2727,Feelings,GN0900,Various Artists,0,Feelings nothing more than feelings Trying to ...,4639910
1,2,GN0301,20070625,Sexualmetro,353020,224583,Lovers’ Leap (Feat. Qypthone),GN0300,애플스,8,i feel alright now but don't know how to speak...,1657318
2,3,"GN0105,GN0101",20170320,Pastel Reflection,10047088,753752,"사랑, 그대라는 멜로디",GN0100,진호,9,그대 스치는 바람처럼 불어와서 내 곁에 머무른 사람 나도 몰래 내쉬는 숨처럼 익숙해...,30310140
3,7,"GN2503,GN0205,GN2501,GN2506,GN0201",20160226,Melting,2669407,750053,Girl Crush,"GN2500,GN0200",마마무 (Mamamoo),17,이따 거기서 봐 이번엔 장담해 찾았어 Hot place 예감이 괜찮아 정말로 기대돼...,8068890
4,8,"GN0805,GN0501,GN0502,GN0801,GN0509",20150205,내가 부른 그림 2,2303168,230399,무얼 기다리나 (Feat. 조원선),"GN0500,GN0800",이영훈,19,그냥 생각 없이 이렇다 할 뜻도 없이 쉼 없이 웃으며 떠드는 이들을 가만히 두리번거...,5579388
...,...,...,...,...,...,...,...,...,...,...,...,...
206928,99957,GN0701,19960101,박윤경 카페클럽 2,327646,5353,장녹수,GN0700,박윤경,153522,가는 세월 바람 타고 흘러가는 저 구름아 수많은 사연 담아 가는 곳이 어드메냐 구중...,1455179
206929,99970,GN0101,19940600,New Brand Spice,308301,100950,다시 비가와,GN0100,코나,153545,누구도 원했던건 아니지만 어느새 시간은 여기까지 우릴 데리고 왔지 어떻게 지내왔는지...,933221
206930,99972,GN0901,20020801,CF Collection Gold,24392,631,Astrud,GN0900,Basia,153548,Nobody knows where she came from The tall-and-...,337742
206931,99979,"GN2003,GN2001",20120203,Eros Best Love Songs,2075386,2261,Anche Tu,GN2000,Eros Ramazzotti,153556,Cissa' se c'e` un angelo per chi non sa non pu...,3697367


### 장르 추가 제거

In [20]:
genre = pd.read_csv("genre_gn_all.tsv",sep ='\t')

In [21]:
instrumental_music = ['GN1100', 'GN1600', 'GN1700', 'GN1800', 'GN1900', 'GN2000', 'GN2100', 'GN2200', 'GN2300', 'GN2400', 'GN2600', 'GN2700', 'GN2800','GN2900', 'GN3000']
main_genre = genre[genre['genre_id'].str.contains('^.*00$', regex=True)]
# Omitting the genre of instrumental music 
ss_lyric_genre = main_genre[~main_genre['genre_id'].isin(instrumental_music)]
meta = meta[ np.vectorize(lambda x : bool(set(x.split(',')) & set(ss_lyric_genre['genre_id']) ))(meta['song_gn_gnr_basket'])]

In [22]:
meta.reset_index(drop = True, inplace = True)

### 가사 및 노래 중복 제거

In [23]:
meta.drop_duplicates("lyric", keep = False, inplace = True)
meta.drop_duplicates("song_id", keep = False, inplace = True)

C:\Users\user\AppData\Local\Temp\ipykernel_7032\3738303722.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meta.drop_duplicates("lyric", keep = False, inplace = True)
C:\Users\user\AppData\Local\Temp\ipykernel_7032\3738303722.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meta.drop_duplicates("song_id", keep = False, inplace = True)


In [24]:
meta.reset_index(drop = True, inplace = True)
meta

,index,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,lyric,song_id
0,0,GN0901,20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,2727,Feelings,GN0900,Various Artists,0,Feelings nothing more than feelings Trying to ...,4639910
1,2,GN0301,20070625,Sexualmetro,353020,224583,Lovers’ Leap (Feat. Qypthone),GN0300,애플스,8,i feel alright now but don't know how to speak...,1657318
2,3,"GN0105,GN0101",20170320,Pastel Reflection,10047088,753752,"사랑, 그대라는 멜로디",GN0100,진호,9,그대 스치는 바람처럼 불어와서 내 곁에 머무른 사람 나도 몰래 내쉬는 숨처럼 익숙해...,30310140
3,7,"GN2503,GN0205,GN2501,GN2506,GN0201",20160226,Melting,2669407,750053,Girl Crush,"GN2500,GN0200",마마무 (Mamamoo),17,이따 거기서 봐 이번엔 장담해 찾았어 Hot place 예감이 괜찮아 정말로 기대돼...,8068890
4,8,"GN0805,GN0501,GN0502,GN0801,GN0509",20150205,내가 부른 그림 2,2303168,230399,무얼 기다리나 (Feat. 조원선),"GN0500,GN0800",이영훈,19,그냥 생각 없이 이렇다 할 뜻도 없이 쉼 없이 웃으며 떠드는 이들을 가만히 두리번거...,5579388
...,...,...,...,...,...,...,...,...,...,...,...,...
133056,99950,"GN0601,GN0604",19970100,무지개,4018,101013,기타로 오토바이를 타자,GN0600,산울림,153507,기타로 오토바이 타자 기타로 오토바이 타자 기타로 오토바이 타자 타자 오토바이로 기...,91251
133057,99957,GN0701,19960101,박윤경 카페클럽 2,327646,5353,장녹수,GN0700,박윤경,153522,가는 세월 바람 타고 흘러가는 저 구름아 수많은 사연 담아 가는 곳이 어드메냐 구중...,1455179
133058,99970,GN0101,19940600,New Brand Spice,308301,100950,다시 비가와,GN0100,코나,153545,누구도 원했던건 아니지만 어느새 시간은 여기까지 우릴 데리고 왔지 어떻게 지내왔는지...,933221
133059,99972,GN0901,20020801,CF Collection Gold,24392,631,Astrud,GN0900,Basia,153548,Nobody knows where she came from The tall-and-...,337742


In [25]:
meta = meta.iloc[:,1:]
meta.columns = ["sub_genre","release","album","album_id","album_id_basket","song_name","genre","artist","id","lyric","song_id"]

In [26]:
meta = meta[['id','song_id','song_name','lyric','artist','genre','album','album_id','release']]
meta

,id,song_id,song_name,lyric,artist,genre,album,album_id,release
0,0,4639910,Feelings,Feelings nothing more than feelings Trying to ...,Various Artists,GN0900,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,20140512
1,8,1657318,Lovers’ Leap (Feat. Qypthone),i feel alright now but don't know how to speak...,애플스,GN0300,Sexualmetro,353020,20070625
2,9,30310140,"사랑, 그대라는 멜로디",그대 스치는 바람처럼 불어와서 내 곁에 머무른 사람 나도 몰래 내쉬는 숨처럼 익숙해...,진호,GN0100,Pastel Reflection,10047088,20170320
3,17,8068890,Girl Crush,이따 거기서 봐 이번엔 장담해 찾았어 Hot place 예감이 괜찮아 정말로 기대돼...,마마무 (Mamamoo),"GN2500,GN0200",Melting,2669407,20160226
4,19,5579388,무얼 기다리나 (Feat. 조원선),그냥 생각 없이 이렇다 할 뜻도 없이 쉼 없이 웃으며 떠드는 이들을 가만히 두리번거...,이영훈,"GN0500,GN0800",내가 부른 그림 2,2303168,20150205
...,...,...,...,...,...,...,...,...,...
133056,153507,91251,기타로 오토바이를 타자,기타로 오토바이 타자 기타로 오토바이 타자 기타로 오토바이 타자 타자 오토바이로 기...,산울림,GN0600,무지개,4018,19970100
133057,153522,1455179,장녹수,가는 세월 바람 타고 흘러가는 저 구름아 수많은 사연 담아 가는 곳이 어드메냐 구중...,박윤경,GN0700,박윤경 카페클럽 2,327646,19960101
133058,153545,933221,다시 비가와,누구도 원했던건 아니지만 어느새 시간은 여기까지 우릴 데리고 왔지 어떻게 지내왔는지...,코나,GN0100,New Brand Spice,308301,19940600
133059,153548,337742,Astrud,Nobody knows where she came from The tall-and-...,Basia,GN0900,CF Collection Gold,24392,20020801


In [27]:
meta.to_csv("lyric_data_v2.tsv",sep = '\t', index= None)